# 3 basic approaches in Bag of Words
https://towardsdatascience.com/3-basic-approaches-in-bag-of-words-which-are-better-than-word-embeddings-c2cbc7398016

In [1]:
import collections
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [2]:
from sklearn.datasets import fetch_20newsgroups
train_raw_df = fetch_20newsgroups(subset='train')

Counting word occurrence. The reason behind of using this approach is that keyword or important signal will occur again and again. So if the number of occurrence represent the importance of word. More frequency means more importance.

In [3]:
doc = "In the-state-of-art of the NLP field, Embedding is the \
success way to resolve text related problem and outperform \
Bag of Words ( BoW ). Indeed, BoW introduced limitations \
large feature dimension, sparse representation etc."

In [4]:
count_vec = CountVectorizer()
count_occurs = count_vec.fit_transform([doc])

In [10]:
count_occur_df = pd.DataFrame((count, word) for word, count in \
                              zip(count_occurs.toarray().tolist()[0], count_vec.get_feature_names()))

In [14]:
count_occur_df.columns = ['Word', 'Count']
count_occur_df.sort_values('Count', ascending=False, inplace=True)

In [16]:
count_occur_df.head()

,Word,Count
16,of,3
26,the,3
3,bow,2
0,and,1
28,way,1


## Normalized Count Occurrence
High frequency may dominate the result and causing model bias. Normalization can be apply to pipeline easily

In [18]:
! git status

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	bow-model-generation.ipynb

nothing added to commit but untracked files present (use "git add" to track)
